# Jupyter Notebook implementation

First read the [README](README.md) file if you're new.

This is an example of using the code from Jupyter Notebook.

## Code Tree

- `Pointnet_Pointnet2_pytorch/`
  - `data/*`: Default location for data.
  - `data_utils/*`: Data Loader.
  - `models/*`: Model file, `torch.nn.Module` classes. See below for more details.
  - `log/*`: log and output of trained model.
  - `visualize/*`: Visualization code. This is independent of the main code.
  - `test_*.py`: Predeiction codes.
  - `train_*.py`: Training codes.

## Data directory:

This could be named anything and placed anywhere, as long as its path is passed to the argument.

- `data/`: Data directory, create this manually.
  - `modelnet40_normal_resampled/*`: ModelNet40 dataset.
  - `shapenetcore_partanno_segmentation_benchmark_v0_normal/*`: ShapeNet dataset.
  - `Stanford3dDataset_v1.2_Aligned_Version/*`: S3DIS dataset.
  - (`stanford_indoor3d/*`): S3DIS dataset converted by `data_utils/collect_indoor3d_data.py`.

For details, see below sections for each model.

## Models

When running `main()`, the argument `--model` will specify the model to be used.
It will search the directory `models/` for a file name that matches the argument.
Create a new file in the `models/` directory to add a new model.

### Aliases

Core Model

- `pointnet_*.py`: PointNet model
- `pointnet2_*.py`: PointNet++ model

Task Type

- `*_cls*.py`: Classification model
- `*_sem_seg*.py`: Segmentation model
- `*_part_seg*.py`: Part segmentation model

Grouping Method

- `*_msg.py`: Multi-scale grouping model
- `*_ssg.py`: Single-scale grouping model

### Model Directory Tree

- `Pointnet_Pointnet2_pytorch/models/`
  - `pointnet_cls.py`: PointNet classification model
  - `pointnet_part_seg.py`: PointNet part segmentation model
  - `pointnet_sem_seg.py`: PointNet semantic segmentation model
  - `pointnet_utils.py`: PointNet util functions
  - `pointnet2_cls_msg.py`: PointNet++ classification model with multi-scale grouping
  - `pointnet2_cls_ssg.py`: PointNet++ classification model with single-scale grouping
  - `pointnet2_part_seg_msg.py`: PointNet++ part segmentation model with multi-scale grouping
  - `pointnet2_part_seg_ssg.py`: PointNet++ part segmentation model with single-scale grouping
  - `pointnet2_sem_seg_msg.py`: PointNet++ semantic segmentation model with multi-scale grouping
  - `pointnet2_sem_seg_ssg.py`: PointNet++ semantic segmentation model with single-scale grouping
  - `pointnet2_utils.py`: PointNet++ util functions

## Imports

In [1]:
from pprint import pprint
from pathlib import Path

import train_classification
import test_classification

import train_partseg
import test_partseg

import train_semseg
import test_semseg
from data_utils.collect_indoor3d_data import collect_indoor3d_data

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


## **Optional**: External Directory

If you want to put the log and data directories in a different location, you can use the following code.

In [2]:
root_log_dir = str(Path('../log').resolve())
data_dir = str(Path('../data').resolve())
class_data_dir = str(Path(data_dir, 'modelnet40_normal_resampled').resolve())
partseg_data_dir = str(Path(data_dir, 'shapenetcore_partanno_segmentation_benchmark_v0_normal').resolve())
semseg_raw_data_dir = str(Path(data_dir, 's3dis', 'Stanford3dDataset_v1.2_Aligned_Version').resolve())
semseg_data_dir = str(Path(data_dir, 'stanford_indoor3d').resolve())

root_log_dir, data_dir, class_data_dir, partseg_data_dir, semseg_raw_data_dir, semseg_data_dir

('/home/gokitune/Documents/pointnet-cleanup/log',
 '/home/gokitune/Documents/pointnet-cleanup/data',
 '/home/gokitune/Documents/pointnet-cleanup/data/modelnet40_normal_resampled',
 '/home/gokitune/Documents/pointnet-cleanup/data/shapenetcore_partanno_segmentation_benchmark_v0_normal',
 '/home/gokitune/Documents/pointnet-cleanup/data/s3dis/Stanford3dDataset_v1.2_Aligned_Version',
 '/home/gokitune/Documents/pointnet-cleanup/data/stanford_indoor3d')

---

## Classification (ModelNet)


### Data

- ModelNet40 dataset in text format
- Download `modelnet40_normal_resampled` from:
  - [Kaggle](https://www.kaggle.com/datasets/chenxaoyu/modelnet-normal-resampled)  
   [Pointcept Huggingface](https://huggingface.co/datasets/Pointcept/modelnet40_normal_resampled-compressed)

Default `--data_dir` is `'data/modelnet40_normal_resampled'`.

- `data/`
  - `modelnet40_normal_resampled/`
    - `train/`
      - `airplane/`
        - `airplane_0001.txt`
        - `airplane_0002.txt`
        - ...
    - ...
    - `file_list.txt`
    - `modelnet10_shape_names.txt`
    - `modelnet10_train.txt`
    - `modelnet10_test.txt`
    - `modelnet40_shape_names.txt`
    - `modelnet40_train.txt`
    - `modelnet40_test.txt`

### **OUTPUT**

- TRAINING: `<log_root>/classification/<args.log_dir or TIME>/checkpoints/best_model.pth`
- PREDICTION: `None (only prints accuracy)`


### Training

`train_classification.py` is used to train the model.

Check all the arguments:

In [4]:
!python train_classification.py -h

usage: training [-h] [--use_cpu] [--gpu GPU] [--batch_size BATCH_SIZE]
                [--model MODEL] [--num_category {10,40}] [--epoch EPOCH]
                [--learning_rate LEARNING_RATE] [--num_point NUM_POINT]
                [--optimizer OPTIMIZER] [--log_root LOG_ROOT]
                [--log_dir LOG_DIR] [--decay_rate DECAY_RATE] [--use_normals]
                [--process_data] [--use_uniform_sample] [--data_dir DATA_DIR]

options:
  -h, --help            show this help message and exit
  --use_cpu             use cpu mode
  --gpu GPU             specify gpu device
  --batch_size BATCH_SIZE
                        batch size in training
  --model MODEL         model name [default: pointnet_cls]
  --num_category {10,40}
                        training on ModelNet10/40
  --epoch EPOCH         number of epoch in training
  --learning_rate LEARNING_RATE
                        learning rate in training
  --num_point NUM_POINT
                        Point Number
  --optimizer OPTI

The following is the same as running:

```bash
python3 train_classification.py \
     --model pointnet2_cls_ssg \
     --log_dir pointnet2_cls_ssg \
     # --log_root ../log \
     # --data_dir ../data/modelnet40_normal_resampled
```

In [ ]:
# user defined arguments for command line as a dictionary
args = {
    'model'   : 'pointnet2_cls_ssg',
    'log_dir' : 'pointnet2_cls_ssg',
    'log_root': root_log_dir,
    'data_dir': class_data_dir,
}
classification_train_args = train_classification.CommandLineArgs(**args)
train_classification.main(classification_train_args)

### Testing

`test_classification.py` is used to test the model.

Check all the arguments:

In [6]:
!python test_classification.py -h

usage: Testing [-h] [--use_cpu] [--gpu GPU] [--batch_size BATCH_SIZE]
               [--num_category {10,40}] [--num_point NUM_POINT]
               [--log_root LOG_ROOT] --log_dir LOG_DIR [--use_normals]
               [--use_uniform_sample] [--num_votes NUM_VOTES]
               [--data_dir DATA_DIR]

options:
  -h, --help            show this help message and exit
  --use_cpu             use cpu mode
  --gpu GPU             specify gpu device
  --batch_size BATCH_SIZE
                        batch size in training
  --num_category {10,40}
                        training on ModelNet10/40
  --num_point NUM_POINT
                        Point Number
  --log_root LOG_ROOT   Log directory root [default: log]
  --log_dir LOG_DIR     Experiment root within log directory
  --use_normals         use normals
  --use_uniform_sample  use uniform sampiling
  --num_votes NUM_VOTES
                        Aggregate classification scores with voting
  --data_dir DATA_DIR   data directory [default:

The following is the same as running:

```bash
python3 test_classification.py \
     --log_dir pointnet2_cls_ssg \
     # --log_root ../log \
     # --data_dir ../data/modelnet40_normal_resampled
```

In [4]:
args = {
    'log_dir': 'pointnet2_cls_ssg',
    'log_root': root_log_dir,
    'data_dir': class_data_dir,
}
classification_test_args = test_classification.CommandLineArgs(**args)
test_classification.main(classification_test_args)

PARAMETER ...
CommandLineArgs(log_dir='pointnet2_cls_ssg', log_root='/home/gokitune/Documents/pointnet-cleanup/log', data_dir='/home/gokitune/Documents/pointnet-cleanup/data/modelnet40_normal_resampled', use_cpu=False, gpu='0', batch_size=24, num_category=40, num_point=1024, use_normals=False, use_uniform_sample=False, num_votes=3)
Load dataset ...
The size of test data is 2468


100%|██████████| 103/103 [00:55<00:00,  1.85it/s]

Test Instance Accuracy: 0.918204, Class Accuracy: 0.883004


---

## Part Segmentation (ShapeNet)

## Data

- ShapeNet dataset in text format
- Download `shapenetcore_partanno_segmentation_benchmark_v0_normal` from:
  - [Kaggle](https://www.kaggle.com/datasets/mitkir/shapenet?resource=download)


Default `--data_dir` is `'data/shapenetcore_partanno_segmentation_benchmark_v0_normal'`.

- `data/`
  - `shapenetcore_partanno_segmentation_benchmark_v0_normal/`
    - `02691156/`
      - `1a04e3eab45ca15dd86060f189eb133.txt`
      - `1a04e3eab45ca15dd86060f189eb133.npz`
      - ...
    - ...
    - `train_test_split/`
      - `shuffled_train_filelist.json`
      - `shuffled_val_filelist.json`
      - `shuffled_test_filelist.json`
    - `utils/`
      - `potential_field.py`
      - `som.py`
    - `synsetoffset2category.txt`

### Data structure

From `data/shapenetcore_partanno_segmentation_benchmark_v0_normal/synsetoffset2category.txt`, the folders correspond to the following categories:

- `Airplane`: 02691156
- `Bag`: 02773838
- `Cap`: 02954340
- `Car`: 02958343
- `Chair`: 03001627
- `Earphone`: 03261776
- `Guitar`: 03467517
- `Knife`: 03624134
- `Lamp`: 03636649
- `Laptop`: 03642806
- `Motorbike`: 03790512
- `Mug`: 03797390
- `Pistol`: 03948459
- `Rocket`: 04099429
- `Skateboard`: 04225987
- `Table`: 04379243

For each .txt file within the folder above, 

 - `[i, :]` is the i th point.
 - `[:, 0:3]` is xyz.
 - `[:, 3:6]` is normalized xyz.
 - `[:, 6]` is the segmentation label.

i.e., each row is a point, and the columns are `[x, y, z, nx, ny, nz, label]`.

`--normal` flag will use all x-y-z-nx-ny-nz + label as input. Otherwise, only x-y-z + label will be used.

#### **OUTPUT**

- TRAINING: `<log_root>/part_seg/<args.log_dir or TIME>/checkpoints/best_model.pth`


### Define Segmentation Classes

In [4]:
# shapenet part segmentation
seg_classes = {
    'Earphone'  : [16, 17, 18],
    'Motorbike' : [30, 31, 32, 33, 34, 35],
    'Rocket'    : [41, 42, 43],
    'Car'       : [8, 9, 10, 11],
    'Laptop'    : [28, 29],
    'Cap'       : [6, 7],
    'Skateboard': [44, 45, 46],
    'Mug'       : [36, 37],
    'Guitar'    : [19, 20, 21],
    'Bag'       : [4, 5],
    'Lamp'      : [24, 25, 26, 27],
    'Table'     : [47, 48, 49],
    'Airplane'  : [0, 1, 2, 3],
    'Pistol'    : [38, 39, 40],
    'Chair'     : [12, 13, 14, 15],
    'Knife'     : [22, 23]
}

seg_ids = [seg_id for seg_val_sublist in seg_classes.values() for seg_id in seg_val_sublist]
len(seg_classes), len(seg_ids)

(16, 50)

### Training

`train_partseg.py` is used to train the model.

Check all the arguments:

In [15]:
!python3 train_partseg.py -h

usage: Model [-h] [--model MODEL] [--batch_size BATCH_SIZE] [--epoch EPOCH]
             [--learning_rate LEARNING_RATE] [--gpu GPU]
             [--optimizer OPTIMIZER] [--log_root LOG_ROOT] [--log_dir LOG_DIR]
             [--decay_rate DECAY_RATE] [--npoint NPOINT] [--normal]
             [--step_size STEP_SIZE] [--lr_decay LR_DECAY]
             [--data_dir DATA_DIR]

options:
  -h, --help            show this help message and exit
  --model MODEL         model name [default: pointnet_part_seg]
  --batch_size BATCH_SIZE
                        batch Size during training [default: 16]
  --epoch EPOCH         epoch to run [default: 251]
  --learning_rate LEARNING_RATE
                        initial learning rate [default: 0.001]
  --gpu GPU             specify GPU devices [default: 0]
  --optimizer OPTIMIZER
                        Adam or SGD [default: Adam]
  --log_root LOG_ROOT   Log root directory [default: log]
  --log_dir LOG_DIR     log path wihin log root directory
  --decay

The following is the same as running:

```bash
python3 train_partseg.py \
     --model pointnet2_part_seg_msg \
     --normal \
     --log_dir pointnet2_part_seg_msg \
     # --log_root ../log \
     # --data_dir ../data/shapenetcore_partanno_segmentation_benchmark_v0_normal
```

In [6]:
args = {
    'model'   : 'pointnet2_part_seg_msg',
    'normal'  : True, # in source: action='store_true'
    'log_dir' : 'pointnet2_part_seg_msg',
    'log_root': root_log_dir,
    'data_dir': partseg_data_dir,
}
partseg_train_args = train_partseg.CommandLineArgs(**args)
train_partseg.main(partseg_train_args, seg_classes)

PARAMETER ...
CommandLineArgs(model='pointnet2_part_seg_msg', normal=True, log_dir='pointnet2_part_seg_msg', log_root='/home/gokitune/Documents/pointnet-cleanup/log', data_dir='/home/gokitune/Documents/pointnet-cleanup/data/shapenetcore_partanno_segmentation_benchmark_v0_normal', batch_size=16, epoch=251, learning_rate=0.001, gpu='0', optimizer='Adam', decay_rate=0.0001, npoint=2048, step_size=20, lr_decay=0.5)
The number of training data is: 13998
The number of test data is: 2874
No existing model, starting training from scratch...
Epoch 1 (1/251):
Learning rate:0.001000
BN momentum updated to: 0.100000


100%|██████████| 874/874 [05:28<00:00,  2.66it/s]

Train accuracy is: 0.85684



100%|██████████| 180/180 [00:50<00:00,  3.55it/s]

eval mIoU of Airplane       0.747408
eval mIoU of Bag            0.448730
eval mIoU of Cap            0.366766
eval mIoU of Car            0.630356
eval mIoU of Chair          0.874591
eval mIoU of Earphone       0.541641
eval mIoU of Guitar         0.836028
eval mIoU of Knife          0.836282
eval mIoU of Lamp           0.770340
eval mIoU of Laptop         0.943971
eval mIoU of Motorbike      0.218272
eval mIoU of Mug            0.502299
eval mIoU of Pistol         0.676891
eval mIoU of Rocket         0.236908
eval mIoU of Skateboard     0.621578
eval mIoU of Table          0.773466
Epoch 1 test Accuracy: 0.905572  Class avg mIOU: 0.626595   Inctance avg mIOU: 0.774038
Saving at /home/gokitune/Documents/pointnet-cleanup/log/part_seg/pointnet2_part_seg_msg/checkpoints/best_model.pth
Saving model....
Best accuracy is: 0.90557
Best class avg mIOU is: 0.62660
Best inctance avg mIOU is: 0.77404


{'accuracy': np.float64(0.9055722846533577),
 'class_avg_accuracy': np.float64(0.650499875879649),
 'class_avg_iou': np.float64(0.6265954443050152),
 'inctance_avg_iou': np.float64(0.7740377678676708)}

### Testing

`test_partseg.py` is used to test the model.

Check all the arguments:

In [14]:
!python3 test_partseg.py -h

usage: PointNet [-h] [--batch_size BATCH_SIZE] [--gpu GPU]
                [--num_point NUM_POINT] [--log_root LOG_ROOT] --log_dir
                LOG_DIR [--normal] [--num_votes NUM_VOTES]
                [--data_dir DATA_DIR]

options:
  -h, --help            show this help message and exit
  --batch_size BATCH_SIZE
                        batch size in testing [default: 24]
  --gpu GPU             specify gpu device [default: 0]
  --num_point NUM_POINT
                        point Number [default: 2048]
  --log_root LOG_ROOT   Log directory root [default: log]
  --log_dir LOG_DIR     experiment root within log directory
  --normal              use normals
  --num_votes NUM_VOTES
                        aggregate segmentation scores with voting [default: 3]
  --data_dir DATA_DIR   data directory [default: data/shapenetcore_partanno_se
                        gmentation_benchmark_v0_normal]


The following is the same as running:

```bash
python3 test_partseg.py \
     --normal \
     --log_dir pointnet2_part_seg_msg \
     # --log_root ../log \
     # --data_dir ../data/shapenetcore_partanno_segmentation_benchmark_v0_normal
```

In [5]:
args = {
    'normal'  : True, # in source: action='store_true'
    'log_dir' : 'pointnet2_part_seg_msg',
    'log_root': root_log_dir,
    'data_dir': partseg_data_dir,
}
partseg_test_args = test_partseg.CommandLineArgs(**args)
test_metrics, shape_ious, total_correct_class, total_seen_class = test_partseg.main(partseg_test_args, seg_classes)

PARAMETER ...
CommandLineArgs(normal=True, log_dir='pointnet2_part_seg_msg', log_root='/home/gokitune/Documents/pointnet-cleanup/log', data_dir='/home/gokitune/Documents/pointnet-cleanup/data/shapenetcore_partanno_segmentation_benchmark_v0_normal', batch_size=24, gpu='0', num_point=2048, num_votes=3)
The number of test data is: 2874


100%|██████████| 120/120 [01:52<00:00,  1.07it/s]

eval mIoU of Airplane       0.749901
eval mIoU of Bag            0.448853
eval mIoU of Cap            0.364924
eval mIoU of Car            0.627209
eval mIoU of Chair          0.874184
eval mIoU of Earphone       0.519588
eval mIoU of Guitar         0.837750
eval mIoU of Knife          0.841147
eval mIoU of Lamp           0.770766
eval mIoU of Laptop         0.945162
eval mIoU of Motorbike      0.217305
eval mIoU of Mug            0.499515
eval mIoU of Pistol         0.681637
eval mIoU of Rocket         0.235913
eval mIoU of Skateboard     0.624753
eval mIoU of Table          0.773184
Accuracy is: 0.90576
Class avg accuracy is: 0.65018
Class avg mIOU is: 0.62574
Inctance avg mIOU is: 0.77422


In [18]:
test_metrics, shape_ious

({'accuracy': 0.944225335170929,
  'class_avg_accuracy': 0.8729847674087255,
  'class_avg_iou': 0.8269832436731457,
  'inctance_avg_iou': 0.8550982761395242},
 {'Earphone': 0.7129622178618391,
  'Motorbike': 0.7216161809824075,
  'Rocket': 0.6105638185383957,
  'Car': 0.7788951500254214,
  'Laptop': 0.9554060803508303,
  'Cap': 0.8714980462658769,
  'Skateboard': 0.7683450655494334,
  'Mug': 0.9530338720707381,
  'Guitar': 0.9115735365391211,
  'Bag': 0.8337195079879579,
  'Lamp': 0.851985120236742,
  'Table': 0.8307068789292265,
  'Airplane': 0.8302130795435185,
  'Pistol': 0.8257730856157082,
  'Chair': 0.9058747480369523,
  'Knife': 0.8695655102361629})

In [19]:
seg_correct = dict(zip(range(len(seg_ids)), total_correct_class))
seg_total = dict(zip(range(len(seg_ids)), total_seen_class))

seg_acc = {}
for id, correct_n in seg_correct.items():
    total_n = seg_total[id]
    if total_n == 0:
        seg_acc[id] = 0
    else:
        seg_acc[id] = correct_n / total_n
# print(seg_acc)

seg_class_acc = {}
for cat in seg_classes:
    seg_class_acc[cat] = {}
    for id in seg_classes[cat]:
        seg_class_acc[cat][id] = seg_acc[id]

pprint(seg_class_acc)

{'Airplane': {0: 0.9456879956097434,
              1: 0.916028658386737,
              2: 0.8438012489818083,
              3: 0.8907710048113245},
 'Bag': {4: 0.6681193429433456, 5: 0.9960294289384561},
 'Cap': {6: 0.9896882203081402, 7: 0.7613373055279709},
 'Car': {8: 0.8615010959066406,
         9: 0.7923612022575142,
         10: 0.8433229928355123,
         11: 0.9569071357046454},
 'Chair': {12: 0.9639621398000587,
           13: 0.9567408163020801,
           14: 0.934777435434984,
           15: 0.846165024558956},
 'Earphone': {16: 0.9518459069020867,
              17: 0.953810623556582,
              18: 0.2647272727272727},
 'Guitar': {19: 0.9503452811575139,
            20: 0.9030249231819734,
            21: 0.9874232534673892},
 'Knife': {22: 0.8991684246676152, 23: 0.9662365748165428},
 'Lamp': {24: 0.9089753178758414,
          25: 0.9564545662623026,
          26: 0.9634393063583815,
          27: 0.8362064590346975},
 'Laptop': {28: 0.9853675480873605, 29: 0.97386334

---

## Semantic Segmentation (S3DIS) 


### Data

- S3DIS dataset in text format
- Download `Stanford3dDataset_v1.2.zip` from:
  - [https://cvg-data.inf.ethz.ch/s3dis/](https://cvg-data.inf.ethz.ch/s3dis/)

Default `--data_dir` is `'data/s3dis/Stanford3dDataset_v1.2_Aligned_Version'`.

- `data/`
  - `s3dis/`
    - `ReadMe.txt`
    - `Stanford3dDataset_v1.2_Aligned_Version/`
      - `Area_1/`
        - `conferenceRoom_1/`
          - `Annotations/`
            - `beam_1.txt`
            - ...
          - `conferenceRoom_1.txt`
        - ...
      - ...

### **OUTPUT**

- TRAINING: `<log_root>/semantic_segmentation/<args.log_dir or TIME>/checkpoints/best_model.pth`
- PREDICTION: `None (only prints accuracy)`


### Data preparation

Use `data_utils/collect_indoor3d_data.py` to convert the dataset.

This will create a new directory `data/stanford_indoor3d/` (or whatever you specify with argument).

In [8]:
!python3 data_utils/collect_indoor3d_data.py -h

usage: collect_indoor3d_data.py [-h] [--data_path DATA_PATH]
                                [--output_path OUTPUT_PATH]

options:
  -h, --help            show this help message and exit
  --data_path DATA_PATH
                        Path to the indoor3d data directory. Default:
                        data/s3dis/Stanford3dDataset_v1.2_Aligned_Version
  --output_path OUTPUT_PATH
                        Path where processed data will be saved. Default:
                        data/stanford_indoor3d


```bash
python3 data_utils/collect_inddoor3d_data.py \
    --data_path data/s3dis/Stanford3dDataset_v1.2_Alligned_Version \
    --output_path data/stanford_indoor3d
```

In [4]:
collect_indoor3d_data(
    data_path   = semseg_raw_data_dir,
    output_path = semseg_data_dir
)

/home/gokitune/Documents/pointnet-cleanup/data/s3dis/Stanford3dDataset_v1.2_Aligned_Version/Area_1/conferenceRoom_1/Annotations
/home/gokitune/Documents/pointnet-cleanup/data/s3dis/Stanford3dDataset_v1.2_Aligned_Version/Area_1/conferenceRoom_1/Annotations/door_1.txt
/home/gokitune/Documents/pointnet-cleanup/data/s3dis/Stanford3dDataset_v1.2_Aligned_Version/Area_1/conferenceRoom_1/Annotations/chair_10.txt
/home/gokitune/Documents/pointnet-cleanup/data/s3dis/Stanford3dDataset_v1.2_Aligned_Version/Area_1/conferenceRoom_1/Annotations/chair_3.txt
/home/gokitune/Documents/pointnet-cleanup/data/s3dis/Stanford3dDataset_v1.2_Aligned_Version/Area_1/conferenceRoom_1/Annotations/wall_4.txt
/home/gokitune/Documents/pointnet-cleanup/data/s3dis/Stanford3dDataset_v1.2_Aligned_Version/Area_1/conferenceRoom_1/Annotations/wall_3.txt
/home/gokitune/Documents/pointnet-cleanup/data/s3dis/Stanford3dDataset_v1.2_Aligned_Version/Area_1/conferenceRoom_1/Annotations/chair_7.txt
/home/gokitune/Documents/pointnet-

### Training

`train_semseg.py` is used to train the model.

Check all the arguments:

In [ ]:
!python3 train_semseg.py -h

usage: Model [-h] [--model MODEL] [--batch_size BATCH_SIZE] [--epoch EPOCH]
             [--learning_rate LEARNING_RATE] [--gpu GPU]
             [--optimizer OPTIMIZER] [--log_dir LOG_DIR]
             [--decay_rate DECAY_RATE] [--npoint NPOINT]
             [--step_size STEP_SIZE] [--lr_decay LR_DECAY]
             [--test_area TEST_AREA] [--data_dir DATA_DIR]

options:
  -h, --help            show this help message and exit
  --model MODEL         model name [default: pointnet_sem_seg]
  --batch_size BATCH_SIZE
                        Batch Size during training [default: 16]
  --epoch EPOCH         Epoch to run [default: 32]
  --learning_rate LEARNING_RATE
                        Initial learning rate [default: 0.001]
  --gpu GPU             GPU to use [default: GPU 0]
  --optimizer OPTIMIZER
                        Adam or SGD [default: Adam]
  --log_dir LOG_DIR     Log path [default: None]
  --decay_rate DECAY_RATE
                        weight decay [default: 1e-4]
  --npoint NP

The following is the same as running:

```bash
python3 train_semseg.py \
     --model pointnet2_sem_seg \
     --test_area 5 \
     --log_dir pointnet2_sem_seg
```

In [ ]:
args = {
    'model'    : 'pointnet2_sem_seg',
    'test_area': 5,
    'log_dir'  : 'pointnet2_sem_seg',
    'log_root' : root_log_dir,
    'data_dir' : semseg_data_dir,
}
semseg_train_args = train_semseg.CommandLineArgs(**args)
train_semseg.main(semseg_train_args)

### Testing

`test_semseg.py` is used to test the model.

Check all the arguments:

In [6]:
!python test_semseg.py -h

usage: Model [-h] [--batch_size BATCH_SIZE] [--gpu GPU]
             [--num_point NUM_POINT] --log_dir LOG_DIR [--visual]
             [--test_area TEST_AREA] [--num_votes NUM_VOTES]
             [--data_dir DATA_DIR]

options:
  -h, --help            show this help message and exit
  --batch_size BATCH_SIZE
                        batch size in testing [default: 32]
  --gpu GPU             specify gpu device
  --num_point NUM_POINT
                        point number [default: 4096]
  --log_dir LOG_DIR     experiment root
  --visual              visualize result [default: False]
  --test_area TEST_AREA
                        area for testing, option: 1-6 [default: 5]
  --num_votes NUM_VOTES
                        aggregate segmentation scores with voting [default: 5]
  --data_dir DATA_DIR   data directory [default: data/s3dis/stanford_indoor3d]


The following is the same as running:

```bash
python3 test_semseg.py \
     --log_dir pointnet2_sem_seg \
     --test_area 5 \
     --visual \
     # --log_root ../log \
     # --data_dir ../data/stanford_indoor3d
```

In [3]:
args = {
    'log_dir'  : 'pointnet2_sem_seg',
    'test_area': 5,
    'visual'   : True, # in source: action='store_true'
    'log_root' : root_log_dir,
    'data_dir' : semseg_data_dir,
}
semseg_test_args = test_semseg.CommandLineArgs(**args)
test_semseg.main(semseg_test_args)

PARAMETER ...
CommandLineArgs(log_dir='pointnet2_sem_seg', test_area=5, visual=True, log_root='/home/gokitune/Documents/pointnet-cleanup/log', data_dir='/home/gokitune/Documents/pointnet-cleanup/data/stanford_indoor3d', batch_size=32, gpu='0', num_point=4096, num_votes=3)
The number of test data is: 67
---- EVALUATION WHOLE SCENE----
Inference [1/67] Area_5_conferenceRoom_2 ...


100%|██████████| 3/3 [02:05<00:00, 41.76s/it]


[0.9930258  0.96915428 0.73365378 0.         0.         0.
 0.19706284 0.90202973 0.81177384 0.88107863 0.76275067 0.53613413
 0.55507851]
Mean IoU of Area_5_conferenceRoom_2: 0.6674
----------------------------
Inference [2/67] Area_5_office_20 ...


100%|██████████| 3/3 [00:23<00:00,  7.90s/it]


[9.76788628e-01 9.75623067e-01 7.26957135e-01 0.00000000e+00
 1.00512614e-04 0.00000000e+00 1.13180764e-01 8.73548190e-01
 8.99098535e-01 0.00000000e+00 5.07601766e-01 0.00000000e+00
 2.93731189e-01]
Mean IoU of Area_5_office_20: 0.5963
----------------------------
Inference [3/67] Area_5_office_26 ...


...

[0.96865528 0.98923676 0.59655759 0.         0.         0.
 0.2025377  0.         0.         0.         0.         0.
 0.60465031]
Mean IoU of Area_5_WC_2: 0.6723
----------------------------
Inference [66/67] Area_5_office_23 ...


100%|██████████| 3/3 [00:42<00:00, 14.32s/it]


[0.97423103 0.97080383 0.617047   0.         0.07545021 0.70735767
 0.03515897 0.76973342 0.86969308 0.         0.86695293 0.54915286
 0.55925317]
Mean IoU of Area_5_office_23: 0.6359
----------------------------
Inference [67/67] Area_5_office_19 ...


100%|██████████| 3/3 [01:41<00:00, 33.92s/it]


[0.94213884 0.98881074 0.70317241 0.         0.         0.
 0.07137822 0.81243562 0.87562365 0.         0.52446808 0.5825969
 0.39454037]
Mean IoU of Area_5_office_19: 0.6550
----------------------------
------- IoU --------
class ceiling       , IoU: 0.905 
class floor         , IoU: 0.977 
class wall          , IoU: 0.734 
class beam          , IoU: 0.000 
class column        , IoU: 0.081 
class window        , IoU: 0.616 
class door          , IoU: 0.146 
class table         , IoU: 0.689 
class chair         , IoU: 0.792 
class sofa          , IoU: 0.554 
class bookcase      , IoU: 0.587 
class board         , IoU: 0.542 
class clutter       , IoU: 0.429 

eval point avg class IoU: 0.542551
eval whole scene point avg class acc: 0.649303
eval whole scene point accuracy: 0.824602
Done!
